In [1]:
from helpers_jpeg import *
from LBT_huffman import *
from helpers import *
import numpy as np
import json

In [6]:
lighthouse = sio.loadmat('mat/SF2_competition_image_2019.mat')
lighthouse2 = sio.loadmat('mat/lighthouse.mat')
flamingo = sio.loadmat('mat/flamingo.mat')
bridge = sio.loadmat('mat/bridge.mat')
X = np.array(lighthouse['X'], dtype=float)
B = np.array(flamingo['X'], dtype=float)
F = np.array(bridge['X'], dtype=float)
G = np.array(lighthouse2['X'], dtype=float)
images = {
    'competition': X-128, 
    'flamingo': B-128, 
    'bridge': F-128,
    'lighthouse': G-128
    
}

In [15]:
def dwt(X, lo=1000, hi=10000, tol=2, max_iter=100, maxbits=40960, n=3):
    '''Return: compressed image, file size'''
    i = 0
    dwt = DwtHuffmanHelper()
    while i < max_iter:
        mid = (lo + hi) / 2
        vlc, _bits, _huffval = dwt.dwtenc(X, mid, n=n)
        current_bits = sum(vlc[:, 1])
        if sum(vlc[:, 1]) > maxbits:
            lo = mid
        else:
            hi = mid
            if abs(current_bits - maxbits) < tol:
                print('Best qstep:', mid)
                return vlc, current_bits, mid
        i += 1
        print('Step: ', mid, 'Bits:', current_bits)
    if sum(vlc[:, 1]) > maxbits:
        print('Cannot compress under {} bits!'.format(maxbits))
    return vlc, sum(vlc[:, 1]), mid

In [19]:
result = {}
for img in images:
    vlc, bits, qstep = dwt(images[img], lo=1000, hi=10000, tol=300, n=4)
    print('Size: {} bits'.format(bits))
    result[img + 'vlc'] = vlc
    result[img + 'qstep'] = qstep

Step:  5500.0 Bits: 25029
Step:  3250.0 Bits: 51927
Step:  4375.0 Bits: 35884
Step:  3812.5 Bits: 42894
Step:  4093.75 Bits: 39198
Step:  3953.125 Bits: 41057
Step:  4023.4375 Bits: 40139
Step:  3988.28125 Bits: 40633
Best qstep: 3970.703125
Size: 40802 bits
Step:  5500.0 Bits: 36378
Step:  3250.0 Bits: 58882
Step:  4375.0 Bits: 45273
Step:  4937.5 Bits: 40173
Step:  4656.25 Bits: 42855
Step:  4796.875 Bits: 41624
Step:  4867.1875 Bits: 41090
Step:  4902.34375 Bits: 40504
Best qstep: 4884.765625
Size: 40835 bits
Step:  5500.0 Bits: 28725
Step:  3250.0 Bits: 58746
Step:  4375.0 Bits: 41123
Step:  4937.5 Bits: 34619
Step:  4656.25 Bits: 37578
Step:  4515.625 Bits: 39457
Step:  4445.3125 Bits: 40342
Step:  4410.15625 Bits: 40614
Best qstep: 4392.578125
Size: 40922 bits
Step:  5500.0 Bits: 28923
Step:  3250.0 Bits: 47474
Step:  4375.0 Bits: 36587
Step:  3812.5 Bits: 41588
Step:  4093.75 Bits: 38839
Step:  3953.125 Bits: 40222
Best qstep: 3882.8125
Size: 40860 bits


In [20]:
sio.savemat('competition_out/json/dwt4.mat', result)

In [11]:
def lbt16(X, N=16, M=16, lo=0, hi=128, tol=2, max_iter=100, maxbits=40960):
    '''Return: compressed image, file size'''
    i = 0
    lbt = LBTHuffman()
    while i < max_iter:
        mid = (lo + hi) / 2
        vlc, _bits, _huffval = lbt.lbthuffenc(X, mid, N=N, M=M, log=False)
        current_bits = sum(vlc[:, 1])
        if sum(vlc[:, 1]) > maxbits:
            lo = mid
        else:
            hi = mid
            if abs(current_bits - maxbits) < tol:
                print('Best qstep:', mid)
                return vlc, current_bits, mid
        i += 1
        print('Step: ', mid, 'Bits:', current_bits)
    if sum(vlc[:, 1]) > maxbits:
        print('Cannot compress under {} bits!'.format(maxbits))
    return vlc, sum(vlc[:, 1]), mid

In [14]:
result = {}
for img in images:
    vlc, bits, qstep = lbt16(images[img], tol=64, N=4, M=16)
    print('Size: {} bits'.format(bits))
    result[img + 'vlc'] = vlc
    result[img + 'qstep'] = qstep
sio.savemat('competition_out/json/lbt16x4.mat', result)

Step:  64.0 Bits: 31854
Step:  32.0 Bits: 68238
Step:  48.0 Bits: 44539
Step:  56.0 Bits: 37255
Step:  52.0 Bits: 40314
Step:  50.0 Bits: 42334
Step:  51.0 Bits: 41379
Step:  51.5 Bits: 40836
Step:  51.25 Bits: 40965
Best qstep: 51.375
Size: 40905 bits
Step:  64.0 Bits: 47294
Step:  96.0 Bits: 34203
Step:  80.0 Bits: 39678
Step:  72.0 Bits: 42979
Step:  76.0 Bits: 41177
Step:  78.0 Bits: 40540
Step:  77.0 Bits: 40772
Step:  76.5 Bits: 40996
Best qstep: 76.75
Size: 40952 bits
Step:  64.0 Bits: 35758
Step:  32.0 Bits: 76556
Step:  48.0 Bits: 50796
Step:  56.0 Bits: 42539
Step:  60.0 Bits: 39216
Step:  58.0 Bits: 40830
Step:  57.0 Bits: 41504
Step:  57.5 Bits: 41183
Step:  57.75 Bits: 41082
Step:  57.875 Bits: 40970
Step:  57.9375 Bits: 40882
Step:  57.90625 Bits: 40894
Best qstep: 57.890625
Size: 40918 bits
Step:  64.0 Bits: 35474
Step:  32.0 Bits: 60670
Step:  48.0 Bits: 44804
Step:  56.0 Bits: 39825
Step:  52.0 Bits: 41947
Best qstep: 54.0
Size: 40924 bits


In [3]:
def jpeg16(X, N=16, M=16, lo=0, hi=128, tol=2, max_iter = 1000, maxbits=40960):
    '''Return: compressed image, file size'''
    i = 0
    jpg = JpegHuffmanHelper()
    while i < max_iter:
        mid = (lo + hi) / 2
        vlc, _bits, _huffval = jpg.jpegenc(X, mid, N=N, M=M, log=False)
        current_bits = sum(vlc[:, 1])
        if sum(vlc[:, 1]) > maxbits:
            lo = mid
        else:
            hi = mid
            if abs(current_bits - maxbits) < tol:
                print('Best qstep:', mid)
                return vlc, current_bits, mid
        i += 1
        print('Step: ', mid, 'Bits:', current_bits)
    if sum(vlc[:, 1]) > maxbits:
        print('Cannot compress under {} bits!'.format(maxbits))
    return vlc, sum(vlc[:, 1]), mid

In [10]:
result = {}
for img in images:
    vlc, bits, qstep = jpeg16(images[img], N=256,M=256, tol=300)
    print('Size: {} bits'.format(bits))
    result[img + 'vlc'] = vlc
    result[img + 'qstep'] = qstep
sio.savemat('competition_out/json/jpeg256x256.mat', result)

Step:  64.0 Bits: 22298
Step:  32.0 Bits: 64050
Step:  48.0 Bits: 38504
Step:  40.0 Bits: 45126
Step:  44.0 Bits: 40477
Step:  42.0 Bits: 43846
Step:  43.0 Bits: 43298
Step:  43.5 Bits: 42984
Step:  43.75 Bits: 40622
Best qstep: 43.625
Size: 40678 bits


ValueError: DC coefficients too large for desired number of bits